# TIME SERIES tutorial using JAN 2022 TPS data 📈📊

Hi to the community!

In this public notebook, I build a tutorial on time series using both classic and ML based methods. Some of the concepts we will be discussing includes:
- Moving Average
- Weighted Moving Average
- Exponential Smoothing
- Holt model
- Holt-Winter model
- Stationarity
- Feature extraction for time-series
- Boosting models for time-series

I also use the following libraries to the create the models:
- Statsmodels
- Scikit-learn
- XGBoost

**Note: This is a tutorial on time series using the TPS data. Therefore, I do not focus on the full data or a submission. I leverage the TPS data in the simplest way possible to teach/practise a tutorial on time-series**

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:maroon; border:0; color:white' role="tab" aria-controls="home"><center>If you find this notebook useful, do give an upvote before forking it </center></h2>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import statsmodels.api as sm
import statsmodels.tsa.api as smt
from sklearn import preprocessing
from sklearn import metrics
from xgboost import XGBRegressor

In [ ]:
!tree ../input/

We only use the training data from TPS for this tutorial

In [ ]:
df = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv', 
           parse_dates=['date'], 
           index_col = ['date'])
df.head(5)

In [ ]:
print('Country info in the data-\n', df['country'].value_counts(), ' \n')
print('Store info in the data-\n', df['store'].value_counts(), ' \n')
print('Product info in the data-\n', df['product'].value_counts(), ' \n')

Since the data is equally distributed across the various categories in the columns, I am filtering the data for the following values, so that the data is simple for the tutorial:
- Country : Sweden
- Store: KaggleMart
- Product: Kaggle Mug
- Index: Only 1 year of data i.e the year of 2015

In [ ]:
df = df.loc[(df['country']=='Sweden') & (df['store']=='KaggleMart') & (df['product']=='Kaggle Mug')]
df = df.drop(['row_id', 'country', 'store', 'product'], axis=1)
df = df[:'2015-12-31']
df

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df['num_sold'])

# MOVING AVERAGE

Moving average is nothing but the average of last n observations in a rolling manner i.e an average of every 30 data points. In our case, here I take the average of last 30 observations

In [ ]:
def moving_avg(col, last_n):
    '''Calculate of the average of last n observations'''
    
    return np.mean(col[-last_n:])

moving_avg(df, 30)

In [ ]:
moving_avg = df.rolling(window=30).mean()
#Pandas way of calculating the same for every 30 days automatically

plt.figure(figsize=(15, 5))
plt.title("Moving average\n window size = {}".format(30))
plt.plot(moving_avg, label="Rolling Average")
plt.plot(df[30:], label='actual values')
plt.legend(loc='best')

# WEIGHTED MOVING AVERAGE
A slightly more complex approach is possible, which is weighted moving average - the total sums up to 1, where the recent K data points are given more weight

In [ ]:
w_moving_avg = df.rolling(window=30, win_type='cosine').mean()
# The weight comes from the win_type parameter. Check out the official docs for all available parameter options

plt.figure(figsize=(15, 5))
plt.title("Moving average\n window size = {}".format(30))
plt.plot(w_moving_avg, label="Rolling Average")
plt.plot(df[30:], label='actual values')
plt.legend(loc='best')

# EXPONENTIAL SMOOTHING

Instead of weighting the last K values, we weight all the data points. Here as we move back in time, the weights are exponentially decreased. It can be expressed by:

$$\hat{y}_{t} = \alpha \cdot y_t + (1-\alpha) \cdot \hat y_{t-1}$$

Here, aplha is the weight of smoothing factor. It defines how much the last observation affects the current. The smaller the alpha, the more influence the previous data point has --> the statsmodels parameter for this is 'smoothing_level'

In [ ]:
ses = smt.SimpleExpSmoothing(df, 
                            initialization_method='heuristic').fit(smoothing_level=0.3, 
                                                                    optimized=False)

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(df, marker="o", color="orange")
plt.plot(ses.fittedvalues, marker="o", color="blue")
plt.legend(loc='best')

# Holt model

Previously, we only took the smoothing level into account. Now we also take the smoothing trend into account. So the formula is updated as: 

$$\ell_x = \alpha y_x + (1-\alpha)(\ell_{x-1} + b_{x-1})$$

$$b_x = \beta(\ell_x - \ell_{x-1}) + (1-\beta)b_{x-1}$$

$$\hat{y}_{x+1} = \ell_x + b_x$$

Here, beta is the trend factor. The final formula as hown depends on both trend & level

In [ ]:
holt = smt.Holt(endog=df,
            initialization_method='heuristic').fit(smoothing_level=0.3, 
                                                   smoothing_trend =0.9, 
                                                   optimized=False)

In [ ]:
plt.figure(figsize=(15,7))
plt.plot(df, marker="o", color="orange")
plt.plot(holt.fittedvalues, marker="o", color="blue")

# Holt-Winter model

Along with trend & level, we add the seasonal component here. So the formula becomes:

$$\ell_x = \alpha(y_x - s_{x-L}) + (1-\alpha)(\ell_{x-1} + b_{x-1})$$

$$b_x = \beta(\ell_x - \ell_{x-1}) + (1-\beta)b_{x-1}$$

$$s_x = \gamma(y_x - \ell_x) + (1-\gamma)s_{x-L}$$

$$\hat{y}_{x+m} = \ell_x + mb_x + s_{x-L+1+(m-1)modL}$$

Here, gamma is the seasonal component

**I am leaving the implementation of Holt-Winters as exercise for the reader.**

**Even I want to implement holt-winters from scratch in plain python. So expect a separate notebook on it in the near future**

# Stationarity

Stationarity mean that the time-series properties like mean, variance etc stay the same throughout the series. It is easier to make predictions on a time-series if we know that the future properties of the series will same as the current data points

We can check for stationarity of a time series using the *dickey-fuller test*. The resuling p value should be less than 0.05 to reject the null hypothesis i.e to reject that the series is non-stationary

In [ ]:
y = pd.Series(df.num_sold)
p_value = sm.tsa.stattools.adfuller(y)[1]
print(p_value)

Here, the p-value id 0.9, which is greater than 0.05. So the null hypothesis told true & the series is non-stationary

One way to deal with stationarity is to take the log of the series & then take the first difference i.e the difference of the series with itself

(Please breakdown the code in the following cell to see the differencing process in action)

In [ ]:
y = pd.Series(np.log(df["num_sold"])).diff().dropna()
p_value = sm.tsa.stattools.adfuller(y)[1]
print(p_value)

Now, the p-value is 0.002 i.e 2%. Therefore we can reject the null hypothesis & say that the series is stationary

# Feature extraction for time-series

Any ML model needs features, but all we have here is a 1-dimensional feature (which we simplified wantedly). So, based on this 1-d feature, how to get more features? Thats where a few feature extraction techniques come in handy

One such technique is to calculate the lag iteratively for a given range

In [ ]:
data = pd.DataFrame(df.num_sold.copy())
data.columns = ["y"]
for i in range(6, 20):
    data["lag_{}".format(i)] = data.y.shift(i)

data.tail(5)

Another technique is to extract the date properties like weekday, weekend, hour etc as separate features

In [ ]:
data.index = pd.to_datetime(data.index)
data["weekday"] = data.index.weekday
data["is_weekend"] = data.weekday.isin([5, 6]) * 1
data.tail(5)

# Boosting models for time-series

To create and validate an ML model, we need train & test set. 

However, in time-series we cannot use the scikit's learning train_test_split() function as it shuffles the data & thereby messing with the time factor. So to keep the series intact, I have reserved the decemeber month's data as test set & the rest as train set

In [ ]:
train = data[:'2015-11-30'].copy()
test = data['2015-12-01':].copy()

In [ ]:
train.tail(5)

In [ ]:
test.tail(5)

In [ ]:
y_train = train.dropna().y
X_train = train.dropna().drop(["y"], axis=1)

y_test = test.dropna().y
X_test = test.dropna().drop(["y"], axis=1)

In [ ]:
def plot_results(model, X_train = X_train, X_test = X_test):
    
    '''Helper function to plot the actual & predicted values from the model'''
    
    prediction = model.predict(X_test)

    plt.figure(figsize=(15, 7))
    plt.plot(prediction, "g", label="prediction", linewidth=2.0)
    plt.plot(y_test.values, label="actual", linewidth=2.0)
    error = metrics.mean_squared_error(prediction, y_test)
    plt.title("Mean squared error {0:.2f}".format(error))
    plt.legend(loc='best')

The input features of the data here is in different scales with lag, weekday & is_weekend. We need to scale the data before proceeding to create a boosting model

In [ ]:
scaler = preprocessing.StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
xgb = XGBRegressor(verbosity=0)
xgb.fit(X_train_scaled, y_train)

In [ ]:
plot_results(xgb,X_train=X_train_scaled,X_test=X_test_scaled)

There is a huge error in predicting the last 5 days of the month. Others have been predicted decently well

### In Part 2 of the notebook, I will discuss how to optimize the prediction of this XGBoost model & discuss some advance concepts including:
## - Time-series cross validation
## - ARIMA family of models
## - Reading ACF & PACF plots
## - LSTM based time-series
## - Prophet library for time series

# Share your thoughts & feedback in the comments
##### Stay safe & enjoy a happy 2022 everyone. Cheers!